In [4]:
import rasterio
from rasterio.windows import from_bounds
import numpy as np
import pandas as pd

# Abrir raster completo
with rasterio.open("/workspaces/OlmosCity/fotos/ESA_WorldCover_10m_2021_V200_S27W063_Map.tif") as src:
    # Definir bounding box (xmin, ymin, xmax, ymax) en coordenadas WGS84
    bounds = (-75.7, 6.1, -75.3, 6.4)  # Medellín aprox
    
    # Crear ventana
    window = from_bounds(*bounds, transform=src.transform)
    
    # Leer solo esa porción
    subset = src.read(1, window=window)
    
    # Obtener el transform de esa ventana
    transform = src.window_transform(window)

# --- Convertir a DataFrame ---
# Crear arrays de coordenadas (X, Y)
rows, cols = np.meshgrid(
    np.arange(subset.shape[0]), 
    np.arange(subset.shape[1]), 
    indexing="ij"
)

xs, ys = rasterio.transform.xy(transform, rows, cols)

# Pasar a columnas
df = pd.DataFrame({
    "X": np.array(xs).flatten(),
    "Y": np.array(ys).flatten(),
    "Z": subset.flatten()
})

# Eliminar celdas sin datos (NaN o nodata)
df = df.dropna(subset=["Z"])
print(df.head())


Empty DataFrame
Columns: [X, Y, Z]
Index: []
